In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
try:
    plt.rcParams['font.family'] = 'AppleGothic'  # Mac
except:
    try:
        plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows
    except:
        print("⚠️ 한글 폰트 설정 실패. 그래프에서 한글이 깨질 수 있습니다.")

plt.rcParams['axes.unicode_minus'] = False

class TumblerAnalyzer:
    def __init__(self, csv_file):
        """CSV 파일 로드"""
        print("="*60)
        print("📂 데이터 로딩 중...")
        print("="*60)
        
        try:
            self.df = pd.read_csv(csv_file, encoding='utf-8-sig')
            print(f"✅ 파일 로드 성공: {csv_file}")
            print(f"📊 초기 데이터: {len(self.df):,}행 x {len(self.df.columns)}열")
            
            # 컬럼 확인
            print(f"\n컬럼 목록: {list(self.df.columns)}")
            
            self.clean_data()
            
        except FileNotFoundError:
            print(f"❌ 파일을 찾을 수 없습니다: {csv_file}")
            print("현재 디렉토리의 CSV 파일을 확인해주세요.")
            return
        except Exception as e:
            print(f"❌ 에러 발생: {e}")
            return
    
    def clean_data(self):
        """데이터 전처리"""
        print("\n" + "="*60)
        print("🧹 데이터 전처리 시작")
        print("="*60)
        
        # 중복 제거
        before = len(self.df)
        if 'product_id' in self.df.columns:
            self.df = self.df.drop_duplicates(subset=['product_id'])
            print(f"✓ 중복 제거: {before:,} → {len(self.df):,} (-{before - len(self.df):,}개)")
        
        # lprice가 문자열인 경우 정수로 변환
        if self.df['lprice'].dtype == 'object':
            self.df['lprice'] = pd.to_numeric(self.df['lprice'], errors='coerce')
            print(f"✓ 가격 데이터 타입 변환 완료")
        
        # 가격이 0이거나 NaN인 데이터 제거
        before = len(self.df)
        self.df = self.df[self.df['lprice'].notna() & (self.df['lprice'] > 0)]
        print(f"✓ 가격 없는 데이터 제거: {before:,} → {len(self.df):,} (-{before - len(self.df):,}개)")
        
        # 가격 이상치 제거 (5,000원 미만, 200,000원 초과)
        before = len(self.df)
        self.df = self.df[(self.df['lprice'] >= 5000) & (self.df['lprice'] <= 200000)]
        print(f"✓ 가격 이상치 제거 (5천~20만원): {before:,} → {len(self.df):,} (-{before - len(self.df):,}개)")
        
        # 용량 추출
        self.df['capacity'] = self.df['title'].apply(self.extract_capacity)
        capacity_count = self.df['capacity'].notna().sum()
        print(f"✓ 용량 추출 완료: {capacity_count:,}개 ({capacity_count/len(self.df)*100:.1f}%)")
        
        # 브랜드 분류
        self.df['brand_type'] = self.df['brand'].apply(self.categorize_brand)
        print(f"✓ 브랜드 분류 완료")
        
        # 가격대 구간
        self.df['price_range'] = pd.cut(self.df['lprice'], 
                                         bins=[0, 10000, 20000, 30000, 50000, 200000],
                                         labels=['1만원 이하', '1-2만원', '2-3만원', '3-5만원', '5만원 이상'])
        
        print(f"\n✅ 전처리 완료: 최종 {len(self.df):,}개 상품")
        print("="*60)
    
    def extract_capacity(self, title):
        """상품명에서 용량 추출"""
        if pd.isna(title):
            return None
            
        title = str(title)
        
        # 350ml, 500ml, 1L 등
        ml_pattern = re.search(r'(\d+)ml', title, re.IGNORECASE)
        l_pattern = re.search(r'(\d+\.?\d*)L', title, re.IGNORECASE)
        
        if ml_pattern:
            ml = int(ml_pattern.group(1))
            return ml
        elif l_pattern:
            return int(float(l_pattern.group(1)) * 1000)
        return None
    
    def categorize_brand(self, brand):
        """브랜드 분류"""
        known_brands = ['스탠리', 'stanley', '써모스', 'thermos', '타이거', 
                       '락앤락', '코멧', '빙그레', 'contigo', '콘티고', 
                       'yeti', '예티', 'hydroflask', '하이드로']
        
        if pd.isna(brand) or brand == 'Unknown' or brand == '':
            return '기타/노브랜드'
        
        brand_lower = str(brand).lower()
        for kb in known_brands:
            if kb.lower() in brand_lower:
                return '유명브랜드'
        
        return '기타/노브랜드'
    
    def basic_statistics(self):
        """기본 통계 분석"""
        print("\n" + "="*60)
        print("📊 기본 통계")
        print("="*60)
        
        print(f"\n총 상품 수: {len(self.df):,}개")
        
        print(f"\n💰 가격 통계:")
        print(f"  평균 가격: {self.df['lprice'].mean():,.0f}원")
        print(f"  중앙값: {self.df['lprice'].median():,.0f}원")
        print(f"  최저가: {self.df['lprice'].min():,.0f}원")
        print(f"  최고가: {self.df['lprice'].max():,.0f}원")
        print(f"  표준편차: {self.df['lprice'].std():,.0f}원")
        
        return self.df.describe()
    
    def price_range_analysis(self):
        """가격대별 분석"""
        print("\n" + "="*60)
        print("💰 가격대별 분포")
        print("="*60)
        
        price_dist = self.df['price_range'].value_counts().sort_index()
        
        print()
        for price, count in price_dist.items():
            percentage = (count / len(self.df)) * 100
            bar_length = int(percentage / 2)
            bar = '█' * bar_length
            print(f"{price:15s}: {count:5,}개 ({percentage:5.1f}%) {bar}")
        
        # 시각화
        try:
            plt.figure(figsize=(12, 6))
            ax = price_dist.plot(kind='bar', color='#3498db', edgecolor='black', alpha=0.8)
            plt.title('가격대별 제품 분포', fontsize=18, fontweight='bold', pad=20)
            plt.xlabel('가격대', fontsize=14, fontweight='bold')
            plt.ylabel('제품 수', fontsize=14, fontweight='bold')
            plt.xticks(rotation=45, ha='right')
            
            # 막대 위에 값 표시
            for i, v in enumerate(price_dist.values):
                ax.text(i, v + max(price_dist.values)*0.01, f'{v:,}개', 
                       ha='center', fontweight='bold')
            
            plt.tight_layout()
            plt.savefig('price_distribution.png', dpi=300, bbox_inches='tight')
            print("\n📁 그래프 저장: price_distribution.png")
            plt.close()
        except Exception as e:
            print(f"\n⚠️ 그래프 저장 실패: {e}")
        
        return price_dist
    
    def capacity_analysis(self):
        """용량별 분석"""
        print("\n" + "="*60)
        print("🥤 용량별 분석")
        print("="*60)
        
        # 용량 있는 데이터만
        df_with_capacity = self.df[self.df['capacity'].notna()].copy()
        
        if len(df_with_capacity) == 0:
            print("⚠️ 용량 정보가 있는 데이터가 없습니다.")
            return None
        
        print(f"용량 정보 있는 제품: {len(df_with_capacity):,}개 ({len(df_with_capacity)/len(self.df)*100:.1f}%)")
        
        # 주요 용량대로 그룹화
        def categorize_capacity(cap):
            if cap < 400:
                return '소형 (<400ml)'
            elif cap < 600:
                return '중형 (400-600ml)'
            elif cap < 1000:
                return '대형 (600ml-1L)'
            else:
                return '특대형 (1L+)'
        
        df_with_capacity['capacity_group'] = df_with_capacity['capacity'].apply(categorize_capacity)
        
        # 용량별 통계
        print("\n📊 용량별 통계:")
        capacity_stats = df_with_capacity.groupby('capacity_group').agg({
            'lprice': ['count', 'mean', 'median']
        }).round(0)
        capacity_stats.columns = ['제품수', '평균가격', '중간가격']
        print(capacity_stats.to_string())
        
        # 용량별 평균 가격 시각화
        try:
            cap_price = df_with_capacity.groupby('capacity_group')['lprice'].mean().sort_values()
            
            plt.figure(figsize=(12, 6))
            ax = cap_price.plot(kind='barh', color='#e74c3c', edgecolor='black', alpha=0.8)
            plt.title('용량별 평균 가격', fontsize=18, fontweight='bold', pad=20)
            plt.xlabel('평균 가격 (원)', fontsize=14, fontweight='bold')
            plt.ylabel('용량', fontsize=14, fontweight='bold')
            
            # 막대 끝에 값 표시
            for i, v in enumerate(cap_price.values):
                ax.text(v + max(cap_price.values)*0.01, i, f'{v:,.0f}원', 
                       va='center', fontweight='bold')
            
            plt.tight_layout()
            plt.savefig('capacity_price.png', dpi=300, bbox_inches='tight')
            print("\n📁 그래프 저장: capacity_price.png")
            plt.close()
        except Exception as e:
            print(f"\n⚠️ 그래프 저장 실패: {e}")
        
        return df_with_capacity
    
    def brand_analysis(self):
        """브랜드 분석"""
        print("\n" + "="*60)
        print("🏷️ 브랜드 분석")
        print("="*60)
        
        brand_stats = self.df.groupby('brand_type').agg({
            'lprice': ['count', 'mean', 'median']
        }).round(0)
        brand_stats.columns = ['제품수', '평균가격', '중간가격']
        
        print("\n📊 브랜드 유형별 통계:")
        print(brand_stats.to_string())
        
        # 브랜드 프리미엄 계산
        known_data = self.df[self.df['brand_type'] == '유명브랜드']['lprice']
        unknown_data = self.df[self.df['brand_type'] == '기타/노브랜드']['lprice']
        
        if len(known_data) > 0 and len(unknown_data) > 0:
            known_avg = known_data.mean()
            unknown_avg = unknown_data.mean()
            premium = ((known_avg - unknown_avg) / unknown_avg) * 100
            
            print(f"\n💎 브랜드 프리미엄 분석:")
            print(f"   유명브랜드 평균: {known_avg:,.0f}원 ({len(known_data):,}개)")
            print(f"   기타브랜드 평균: {unknown_avg:,.0f}원 ({len(unknown_data):,}개)")
            print(f"   브랜드 프리미엄: {premium:.1f}% (유명브랜드가 {premium:.1f}% 더 비쌈)")
        
        # 상위 브랜드 분석
        print(f"\n🔝 상위 브랜드 TOP 10:")
        top_brands = self.df['brand'].value_counts().head(10)
        for i, (brand, count) in enumerate(top_brands.items(), 1):
            print(f"   {i:2d}. {brand:20s}: {count:4,}개")
        
        return brand_stats
    
    def mall_analysis(self):
        """쇼핑몰 분석"""
        print("\n" + "="*60)
        print("🛒 주요 판매처 분석")
        print("="*60)
        
        top_malls = self.df['mall_name'].value_counts().head(15)
        
        print(f"\n상위 15개 쇼핑몰:")
        for i, (mall, count) in enumerate(top_malls.items(), 1):
            percentage = (count / len(self.df)) * 100
            bar_length = int(percentage / 2)
            bar = '█' * bar_length
            print(f"{i:2d}. {mall:25s}: {count:4,}개 ({percentage:5.1f}%) {bar}")
        
        return top_malls
    
    def keyword_analysis(self):
        """인기 키워드 분석"""
        print("\n" + "="*60)
        print("🔍 인기 키워드 분석")
        print("="*60)
        
        # 제목에서 키워드 추출
        all_words = []
        for title in self.df['title']:
            if pd.notna(title):
                words = str(title).split()
                all_words.extend(words)
        
        # 불용어 제거
        stop_words = ['텀블러', '보온병', '물병', 'ml', 'L', '/', '-', '|', 
                     '(', ')', '[', ']', '<b>', '</b>', 'b']
        filtered_words = [w for w in all_words 
                         if w not in stop_words and len(w) > 1 and not w.isdigit()]
        
        # 빈도수 계산
        word_freq = Counter(filtered_words).most_common(30)
        
        print(f"\n🔥 TOP 30 인기 키워드:")
        for i, (word, freq) in enumerate(word_freq, 1):
            print(f"{i:2d}. {word:15s}: {freq:5,}회")
        
        return word_freq
    
    def find_sweet_spot(self):
        """스위트스팟 찾기 (진입 추천 가격대)"""
        print("\n" + "="*60)
        print("🎯 시장 진입 전략 (스위트스팟)")
        print("="*60)
        
        # 가격대별 경쟁 밀도
        price_bins = [0, 10000, 15000, 20000, 25000, 30000, 50000, 200000]
        price_labels = ['~1만', '1-1.5만', '1.5-2만', '2-2.5만', '2.5-3만', '3-5만', '5만+']
        
        self.df['detailed_price'] = pd.cut(self.df['lprice'], bins=price_bins, labels=price_labels)
        
        competition = self.df['detailed_price'].value_counts().sort_index()
        
        print("\n📍 가격대별 제품 수 (경쟁 밀도):")
        max_count = competition.max()
        for price, count in competition.items():
            percentage = (count / len(self.df)) * 100
            intensity = '🔥' * int(count / max_count * 5)
            print(f"{price:10s}: {count:5,}개 ({percentage:5.1f}%) {intensity}")
        
        # 경쟁 적은 구간 찾기
        min_competition = competition.min()
        sweet_spots = competition[competition <= min_competition * 1.5]
        
        print(f"\n💡 추천 진입 가격대 (경쟁 상대적으로 적음):")
        for spot in sweet_spots.index:
            count = competition[spot]
            print(f"   ✓ {spot}원대 ({count:,}개 제품)")
        
        return competition
    
    def generate_report(self):
        """종합 리포트 생성"""
        print("\n" + "="*70)
        print(" "*25 + "📋 종합 분석 리포트")
        print("="*70)
        
        # 모든 분석 실행
        self.basic_statistics()
        self.price_range_analysis()
        self.capacity_analysis()
        self.brand_analysis()
        self.mall_analysis()
        self.keyword_analysis()
        competition = self.find_sweet_spot()
        
        # 최종 추천
        print("\n" + "="*70)
        print(" "*20 + "💰 창업자를 위한 최종 추천")
        print("="*70)
        
        avg_price = self.df['lprice'].mean()
        median_price = self.df['lprice'].median()
        
        print(f"\n1️⃣ 추천 가격 설정:")
        print(f"   • 시장 평균: {avg_price:,.0f}원")
        print(f"   • 중앙값: {median_price:,.0f}원")
        print(f"   • 💡 추천 진입 가격: {int(median_price * 0.9):,}원")
        print(f"      (중앙값의 90% = 가성비 어필 + 경쟁력 확보)")
        
        print(f"\n2️⃣ 추천 제품 스펙:")
        popular_capacity = self.df[self.df['capacity'].notna()]['capacity'].mode()
        if len(popular_capacity) > 0:
            print(f"   • 용량: {int(popular_capacity.values[0])}ml (가장 인기 있는 용량)")
        else:
            print(f"   • 용량: 500ml 추천 (표준 용량)")
        print(f"   • 브랜드: 기타/노브랜드로 시작 (낮은 진입장벽)")
        print(f"   • 차별화: 색상 다양화, 각인 서비스, 독특한 디자인")
        
        print(f"\n3️⃣ 예상 매출 시뮬레이션:")
        recommended_price = int(median_price * 0.9)
        daily_sales = 5
        monthly_revenue = recommended_price * daily_sales * 30
        net_profit = int(monthly_revenue * 0.3)
        
        print(f"   • 판매가: {recommended_price:,}원")
        print(f"   • 하루 판매: {daily_sales}개 (보수적 추정)")
        print(f"   • 월 매출: {monthly_revenue:,}원")
        print(f"   • 예상 순이익: {net_profit:,}원 (이익률 30% 가정)")
        print(f"   • 손익분기점: 약 {int(100000 / (recommended_price * 0.3)):,}개 판매 (초기 투자 10만원 가정)")
        
        print("\n" + "="*70)
        print("✅ 분석 완료!")
        print("="*70)


# ===================
# 실행 코드
# ===================
# CSV 파일 경로 지정
csv_file = "/Users/sebokoh/데이터분석연습/프로젝트(데이터분석3가지)/naver_shopping_20251030_1418.csv"  

# 분석 실행
analyzer = TumblerAnalyzer(csv_file)

# 종합 리포트 생성
analyzer.generate_report()

# 분석된 데이터 저장
output_file = "tumbler_analyzed.csv"
analyzer.df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"\n💾 분석 데이터 저장: {output_file}")

print("\n" + "="*70)
print("🎉 모든 분석이 완료되었습니다!")
print("="*70)
print("\n📊 생성된 파일:")
print("   1. price_distribution.png - 가격대별 분포 그래프")
print("   2. capacity_price.png - 용량별 가격 그래프")
print("   3. tumbler_analyzed.csv - 분석된 데이터")
print("="*70)

📂 데이터 로딩 중...
✅ 파일 로드 성공: /Users/sebokoh/데이터분석연습/프로젝트(데이터분석3가지)/naver_shopping_20251030_1418.csv
📊 초기 데이터: 700행 x 14열

컬럼 목록: ['title', 'link', 'image', 'lprice', 'hprice', 'mall_name', 'product_id', 'brand', 'maker', 'category1', 'category2', 'category3', 'category4', 'search_keyword']

🧹 데이터 전처리 시작
✓ 중복 제거: 700 → 700 (-0개)
✓ 가격 없는 데이터 제거: 700 → 700 (-0개)
✓ 가격 이상치 제거 (5천~20만원): 700 → 676 (-24개)
✓ 용량 추출 완료: 561개 (83.0%)
✓ 브랜드 분류 완료

✅ 전처리 완료: 최종 676개 상품

                         📋 종합 분석 리포트

📊 기본 통계

총 상품 수: 676개

💰 가격 통계:
  평균 가격: 33,647원
  중앙값: 32,365원
  최저가: 5,000원
  최고가: 129,900원
  표준편차: 17,749원

💰 가격대별 분포

1만원 이하         :    34개 (  5.0%) ██
1-2만원          :   142개 ( 21.0%) ██████████
2-3만원          :   125개 ( 18.5%) █████████
3-5만원          :   293개 ( 43.3%) █████████████████████
5만원 이상         :    82개 ( 12.1%) ██████

📁 그래프 저장: price_distribution.png

🥤 용량별 분석
용량 정보 있는 제품: 561개 (83.0%)

📊 용량별 통계:
                제품수     평균가격     중간가격
capacity_group     